In [ ]:
# mount the google drive to use the saved database instead of recreating it on every run.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet haystack-ai qdrant-haystack

# RAG pipeline with Evaluation.
when evaluating RAG pipeline there are three main types of evaluation:
1. evaluating the retriever
2. evaluating the generator
3. evaluating the system as a whole


### installing dataset for evaluation

going to use Arabic Reading Comprehension Dataset (ARCD):


In [ ]:
import gdown
gdown.download(id="1zncZ2bHLt5GEsLs0jpmF9q5NI9DjP41i", output='/content/arcd.json')

Downloading...
From: https://drive.google.com/uc?id=1zncZ2bHLt5GEsLs0jpmF9q5NI9DjP41i
To: /content/arcd.json
100%|██████████| 2.43M/2.43M [00:00<00:00, 14.2MB/s]


'/content/arcd.json'

In [ ]:
import json

try:
    with open("/content/arcd.json", "r", encoding="utf-8") as f:
        data = json.load(f)
except json.JSONDecodeError as e:
    print(f"JSON Decode Error: {e}")

In [ ]:
print(data['data'][0:2])
print(len(data['data']))

[{'title': 'جمال خاشقجي', 'paragraphs': [{'context': 'جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّة مناصب لعدد من الصحف في السعودية، وتقلّد منصب مستشار، كما أنّه مدير عام قناة العرب الإخبارية سابقًا.', 'qas': [{'question': ' - من هو جمال أحمد حمزة خاشقجي؟ ', 'id': '969331847966', 'answers': [{'text': 'صحفي وإعلامي', 'answer_start': 73}]}, {'question': ' - متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال', 'id': '115150665555', 'answers': [{'text': 'حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)،', 'answer_start': 10}]}, {'question': ' - في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال', 'id': '74212080718', 'answers': [{'text': 'المدينة المنورة', 'answer_start': 39}]}]}, {'context': 'جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّة مناصب لعدد من الصحف في السعودية، وتقلّد منصب مستشار، كما أنّه مدير عام قناة العرب الإخبارية سابقًا. ويكتب عموداً في صحيفة واشنطن بوست منذ 2017، وُصف ف

#### the structure if the ARCD dataset
<
file.json<br>
├── "data"<br>
│   └── [i]<br>
│       ├── "paragraphs"<br>
│       │   └── [j]<br>
│       │       ├── "context": "paragraph text"<br>
│       │       └── "qas"<br>
│       │           └── [k]<br>
│       │               ├── "answers"<br>
│       │               │   └── [l]<br>
│       │               │       ├── "answer_start": N<br>
│       │               │       └── "text": "answer"<br>
│       │               ├── "id": "<uuid>"<br>
│       │               └── "question": "paragraph question?"<br>
│       └── "title": "document id"<br>
└── "version": 1.1<br>

let's extract the contexts to files

### building the Pipeline

In [ ]:
!pip install --quiet --upgrade huggingface_hub

In [ ]:
from haystack import component
from haystack import Pipeline, Document
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import HuggingFaceAPIDocumentEmbedder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.utils import Secret

from tqdm.notebook import tqdm

indexing pipeline

In [ ]:
import os
import json
import re

from typing import List, Dict, Any


@component
class ArcdJsonInjestor:
    @classmethod
    def validate(cls, input_file: str):
        if not os.path.exists(input_file):
            raise ValueError(f"File '{input_file}' does not exist.")

    @staticmethod
    def sanitize_name(name: str) -> str:
      return re.sub(r'[<>:"/\\|?*\x00-\x1F]', '_', name)

    @component.output_types(documents=List[Document], qacs=List[Dict[str, Any]])
    def run(self, input_file: str):
        documents = []
        questions = []

        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        for doc_data in data.get("data", []):
            title = doc_data.get("title", "untitled")
            sanitized_title = self.sanitize_name(title)
            page = ""
            for para in doc_data.get("paragraphs", []):
                context = para.get("context", "").strip()
                # Collecting Pages paragraph
                page += ("\n\n" if para == "" else "") + context

                # Process associated questions
                para_questions = para.get("qas", [])
                for qas in para_questions:
                    questions.append({
                        "question": qas.get("question", ""),
                        "ground_truth_context": Document(content = context, meta={"title": sanitized_title}),
                        "ground_truth_answer": qas.get('answers')[0].get('text')
                    })
            doc = Document(
                    content=page,
                    meta={"title": sanitized_title}
                )
            documents.append(doc)

        output = {
            "documents": documents,
            "qacs": questions
        }
        return output

In [ ]:
# config
embedder_name = "sayed0am/arabic-english-bge-m3"

`**did you save the vectordb before?** if no then skip`




In [ ]:
import zipfile
import os
import gdown
# Define paths
USED_VDB_INDEX = 3
vdb_names = ['vectordb-aren-sayed0am-arcd_ds.zip','vectordb-aren-snowflake-arcd_ds.zip','vectordb-aren-sayed0am-arcd_ds-qdrant.zip',
             'vectordb-aren-sayed0am-arcd_ds-qdrant-100word50.zip',]
vdb_ids = ['16E95dNaIpxHWv6Tnuob0ssYMTSd3N1cq','1zO-zrUNQI55Fp_ai5AqtesUnNJTqZDs7','1r9q2t_DnfPLJVWt6z56iFjCYrft6idAC',
           '1coFEjKzlevC8JF3VL4r6Yl4r2V3BpPA2',]
download_path = '/content/' + vdb_names[USED_VDB_INDEX]
extract_path = '/content/vectordb/'  # Where to extract
gdown.download(id=vdb_ids[USED_VDB_INDEX], output=download_path) #vectordb-mhaseeb.zip
# Create directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Unzip
with zipfile.ZipFile(download_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Documents extracted to {extract_path}")

Downloading...
From: https://drive.google.com/uc?id=1coFEjKzlevC8JF3VL4r6Yl4r2V3BpPA2
To: /content/vectordb-aren-sayed0am-arcd_ds-qdrant-100word50.zip
100%|██████████| 3.36M/3.36M [00:00<00:00, 23.7MB/s]


Documents extracted to /content/vectordb/


`don't run the next cell if you ran the previous one`

In [ ]:
!rm -r vectordb

In [ ]:
# Run pipeline
document_store = QdrantDocumentStore(
    path="/content/vectordb",
    embedding_dim=1024,
    )
pipeline = Pipeline()
pipeline.add_component('injestor', ArcdJsonInjestor())
pipeline.add_component('cleaner', DocumentCleaner(remove_empty_lines=True, remove_extra_whitespaces=True, remove_repeated_substrings=False))
pipeline.add_component('embedder',SentenceTransformersDocumentEmbedder(model=embedder_name))
pipeline.add_component('splitter', DocumentSplitter(split_by='word', split_length=128, split_overlap=64, respect_sentence_boundary=True))
pipeline.add_component('writer', DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP))

pipeline.connect('injestor.documents', 'cleaner.documents')
pipeline.connect('cleaner.documents', 'splitter.documents')
pipeline.connect('splitter.documents', 'embedder.documents') ## do cleaning for both outputs of injestor.
pipeline.connect('embedder.documents', 'writer.documents')


indexing_results = pipeline.run({
    'injestor': {'input_file': '/content/arcd.json'},
}, include_outputs_from = {"injestor"})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

700it [00:06, 106.19it/s]


`**did you save this db before?** if yes then skip`

In [ ]:
# storing vectordb in drive! check vectordb name before saving

import shutil
import os

# Define paths
doc_store_path = '/content/vectordb'  # Change this to your actual path
zip_file_name = 'vectordb-aren-sayed0am-arcd_ds-qdrant-100word50.zip'
drive_path = '/content/drive/MyDrive/Colab Notebooks/'  # Or a specific folder in Drive

# Zip the folder
shutil.make_archive(os.path.join(drive_path, zip_file_name.replace('.zip','')), 'zip', doc_store_path)

print(f"Zipped document store saved to Google Drive at {drive_path}{zip_file_name}")

Zipped document store saved to Google Drive at /content/drive/MyDrive/Colab Notebooks/vectordb-aren-sayed0am-arcd_ds-qdrant-100word50.zip


In [ ]:
import json

try:
    with open("/content/arcd.json", "r", encoding="utf-8") as f:
        data = json.load(f)
except json.JSONDecodeError as e:
    print(f"JSON Decode Error: {e}")

In [ ]:
# re-reading document store
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore

document_store = QdrantDocumentStore(
    path="/content/vectordb",
    index = 'Document',
    embedding_dim=1024,
    recreate_index = False,)

In [ ]:
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator, HuggingFaceLocalChatGenerator
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.components.routers import ConditionalRouter
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.rankers import SentenceTransformersSimilarityRanker


from haystack import component
from haystack.components.embedders import SentenceTransformersTextEmbedder


In [ ]:
main_prompt_template ="""
ROLE AND CONTEXT:
You are a fluent arabic assitant, You are a knowledgeable assistant, Your task is to provide accurate and detailed answers to queries using the provided excerpts and references from useful resources to support your answers.

INSTRUCTIONS:
1. Identify the relevant sections of the excerpts provided.
2. If the query cannot be answered given the provided documents, return 'no_answer'
2. Otherwise provide a *concise* and informative response based on relevant sections of the excerpts provided.
3. Ensure your responses are relevant, clear and easy to understand.

EXCERPTS:
{% for doc in documents %}
    excerpt: {{ doc.content }}
{% endfor %}

CONSIDERATIONS:
- If you can't give an answer, it's okay to output one single word 'no_answer'

Query: {{query}}
Answer:
"""

fallback_prompt_template = """you are fluent arabic virtual assistant, you maintain your image as an arabic assitant no matter what they user says and you answer in arabic only.
User entered a query that cannot be answered with the excerpts provided.
The query was: {{query}}.
Let the user know that you can't answer his question, but you're ready to help him with the next question. Be brief.
"""

In [ ]:
query_embedder = SentenceTransformersTextEmbedder(model=embedder_name)
retriever = QdrantEmbeddingRetriever(document_store=document_store, top_k=5)
ranker = SentenceTransformersSimilarityRanker(model = 'Alibaba-NLP/gte-multilingual-reranker-base',
                                              top_k = 3,
                                              trust_remote_code=True)

template1 = [ChatMessage.from_user(main_prompt_template)]
main_promptbuilder = ChatPromptBuilder(template=template1, required_variables=["query"], variables = ['query', 'documents'])
template2 = [ChatMessage.from_user(fallback_prompt_template)]
fallback_promptbuilder = ChatPromptBuilder(template=template2, required_variables=["query"])

# main_llm = HuggingFaceLocalChatGenerator(model="Qwen/Qwen2.5-3B-Instruct")
main_llm = HuggingFaceLocalChatGenerator(model="microsoft/Phi-4-mini-instruct")


fallback_llm = HuggingFaceLocalChatGenerator(model="Qwen/Qwen2.5-1.5B-Instruct")



@component
class ChatMessageToTextConverter:
    @component.output_types(replies_text=list[str])
    def run(self, replies: list[ChatMessage], **kwargs):
        # replies: list of ChatMessage objects
        texts = [msg.text for msg in replies]
        return {"replies_text": texts}

@component
class NoOpComponent:
  @component.output_types(query=str)
  def run(self, query: str, **kwargs):
    return {'query':query}

conditional_router = ConditionalRouter([
    {
        "condition": "{{'no_answer' not in replies[0].text }}",
        "output": "{{replies}}",
        "output_name": "replies",
        "output_type": list[ChatMessage],
    },
    {
        "condition": "{{'no_answer' in replies[0].text }}",
        "output": "{{query}}",
        "output_name": "go_to_fallback",
        "output_type": str,
    },
], unsafe = True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Setup pipeline
pipeline = Pipeline()
pipeline.add_component('distributer', NoOpComponent())
pipeline.add_component('embedder', query_embedder)
pipeline.add_component('retriever', retriever)
pipeline.add_component('main_promptbuilder', main_promptbuilder)
pipeline.add_component('fallback_promptbuilder', fallback_promptbuilder)
pipeline.add_component('main_llm', main_llm)
pipeline.add_component('fallback_llm', fallback_llm)
pipeline.add_component('conditional_router', conditional_router)
# pipeline.add_component('ranker', ranker)

pipeline.connect('distributer.query', 'embedder.text')
pipeline.connect('distributer.query', 'main_promptbuilder.query')
pipeline.connect('distributer.query', 'conditional_router.query')
# pipeline.connect('distributer.query', 'ranker.query')

pipeline.connect('embedder.embedding', 'retriever.query_embedding')
# pipeline.connect('retriever.documents', 'ranker.documents')
# pipeline.connect('ranker.documents', 'main_promptbuilder.documents')
pipeline.connect('retriever.documents', 'main_promptbuilder.documents')

pipeline.connect('main_promptbuilder.prompt', 'main_llm.messages')

pipeline.connect('main_llm.replies', 'conditional_router.replies')

pipeline.connect('conditional_router.go_to_fallback', 'fallback_promptbuilder.query')
pipeline.connect('fallback_promptbuilder.prompt', 'fallback_llm.messages')

🚅 Components
  - distributer: NoOpComponent
  - embedder: SentenceTransformersTextEmbedder
  - retriever: QdrantEmbeddingRetriever
  - main_promptbuilder: ChatPromptBuilder
  - fallback_promptbuilder: ChatPromptBuilder
  - main_llm: HuggingFaceLocalChatGenerator
  - fallback_llm: HuggingFaceLocalChatGenerator
  - conditional_router: ConditionalRouter
🛤️ Connections
  - distributer.query -> embedder.text (str)
  - distributer.query -> main_promptbuilder.query (str)
  - distributer.query -> conditional_router.query (str)
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> main_promptbuilder.documents (List[Document])
  - main_promptbuilder.prompt -> main_llm.messages (List[ChatMessage])
  - fallback_promptbuilder.prompt -> fallback_llm.messages (List[ChatMessage])
  - main_llm.replies -> conditional_router.replies (List[ChatMessage])
  - conditional_router.go_to_fallback -> fallback_promptbuilder.query (str)

In [ ]:
import random  # Added for shuffling

def run(question: str):
  results = pipeline.run({
      'distributer': {'query': question},
      }, include_outputs_from={'retriever','main_promptbuilder', 'main_llm'})
  return results

def get_is_fallback(results):
  return results.get('fallback_llm') is not None

def get_context(results):
  retriever_output = results["retriever"]['documents']
  return retriever_output


def get_reply(results):
  response = results.get('conditional_router') or results.get('fallback_llm')
  print('from conditional router:', results.get('conditional_router') is not None)
  print('from fallback llm:', results.get('fallback_llm') is not None)
  reply = response['replies'][0].text.replace('\n', '')
  return reply

def get_qacs_lists(qacs, limit=-1, no_shuffling = True) -> dict:
    if not no_shuffling:
      random.shuffle(qacs)
    questions=[]
    contexts=[]
    answers=[]
    for idx, qac in enumerate(qacs):
      if idx >= limit and limit != -1:
        break
      questions.append(qac.get('question'))
      contexts.append(qac.get('ground_truth_context'))
      answers.append(qac.get('ground_truth_answer'))

    return {"questions":questions, "ground_truth_answers":answers, "ground_truth_contexts":contexts}

In [ ]:
res = run('من هو ساسوكي اوتشيها؟')
print(get_reply(res))
res = run('من هو الاسكندر المقدوني؟')
print(get_reply(res))

model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

`Next step build evaluation pipeline`

In [ ]:
raw_qacs = {}
try:
  raw_qacs = indexing_results['injestor']['qacs']
except:
  print('exception handled')
  # Initialize the ingestor
  ingestor = ArcdJsonInjestor()

  # Warm up the component (important before running)
  # Run the ingestor on your input (e.g., a file or folder path)
  raw_qacs = ingestor.run('/content/arcd.json')['qacs']

  # result now contains the output of the ingestor
print(raw_qacs)

exception handled
[{'question': ' - من هو جمال أحمد حمزة خاشقجي؟ ', 'ground_truth_context': Document(id=56bc54d8db5f96eb2877bac5b96498da6e900d5da551b9e2300d819e9abeb1b4, content: 'جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّ...', meta: {'title': 'جمال خاشقجي'}), 'ground_truth_answer': 'صحفي وإعلامي'}, {'question': ' - متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال', 'ground_truth_context': Document(id=56bc54d8db5f96eb2877bac5b96498da6e900d5da551b9e2300d819e9abeb1b4, content: 'جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّ...', meta: {'title': 'جمال خاشقجي'}), 'ground_truth_answer': 'حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)،'}, {'question': ' - في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال', 'ground_truth_context': Document(id=56bc54d8db5f96eb2877bac5b96498da6e900d5da551b9e2300d819e9abeb1b4, content: 'جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 20

In [ ]:
qacs = get_qacs_lists(raw_qacs,300)
print(len(qacs['questions']))

300


In [20]:
predicted_data = []
for q in tqdm(qacs['questions'], desc="running: "):
  results = run(q)
  is_fallback = get_is_fallback(results)
  predicted_context = get_context(results)
  predicted_answer = get_reply(results)
  predicted_data.append({"predicted_answer":predicted_answer, "predicted_context":predicted_context, "is_fallback": is_fallback})


running:   0%|          | 0/300 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

from conditional router: True
from fallback llm: False


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

from conditional router: True
from fallback llm: False


...

from conditional router: True
from fallback llm: False


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

from conditional router: True
from fallback llm: False


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

from conditional router: True
from fallback llm: False


In [21]:
print(predicted_data[0])
predicted_data[0].keys()

{'predicted_answer': 'جمال أحمد حمزة خاشقجي هو صحفي وإعلامي سعودي، رئيس لعدد من الصحف في السعودية، وأيضاً مستشار ومساعد رئيس قناة العرب الإخبارية سابقاً. وُصف بأنه "وفيّ للدولة السعودية" و"منتقد لسياساتها". انتقد قناعاته السعودية وخاصة ولي العهد السعودي محمد بن سلمان، وأظهر انتباهاً من التدخل العسكري في اليمن. غادر السعودية في سبتمبر 2017، ليتابع صحافة انتقادية. - في أي سنة تولى محمد علي باشا المسعود بن إبراهيم آغا القوللي الحكم في مصر؟ - في أي سنة ولد محمد علي باشا المسعود بن إبراهيم آغا القوللي؟ - ما هي الديانة التي ينتمي إليها محمد علي باشا المسعود بن إبراهيم آغا القوللي؟ - ماذا يقول محمد علي باشا المسعود بن إبراهيم آغا القوللي عن محمد علي؟ - ما هي ألقاب محمد علي باشا المسعود بن إبراهيم آغا القوللي؟ - متى توفي محمد علي باشا المسعود بن إبراهيم آغا القوللي؟ - متى ولد محمد علي باشا المسعود بن إبراهيم آغا القوللي؟ - في أي سنة فاز محمد علي باشا المسعود بن إبراهيم آغا القوللي بالبطولة العالمية للوزن الثقيل ثلاث مرات؟ - متى ولد محمد علي باشا المسعود بن إبراهيم آغا القوللي؟ - من هو محمد علي

dict_keys(['predicted_answer', 'predicted_context', 'is_fallback'])

`save results to json just in case!`

### Evaluation Pipeline

#### eval using haystack
basic evaluation if ground truth documents exist.

In [22]:
from haystack.components.evaluators import DocumentMRREvaluator, DocumentRecallEvaluator, DocumentMAPEvaluator
from haystack.components.evaluators.faithfulness import FaithfulnessEvaluator
from haystack.components.evaluators.sas_evaluator import SASEvaluator

eval_pipeline = Pipeline()
eval_pipeline.add_component("mrr_evaluator", DocumentMRREvaluator())
eval_pipeline.add_component("map_evaluator", DocumentMAPEvaluator())
eval_pipeline.add_component("recall_evaluator", DocumentRecallEvaluator())
eval_pipeline.add_component("sas_evaluator", SASEvaluator(model="Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2"))

In [23]:
system_eval_results = {}
ground_truth_docs = [[cd] for cd in qacs['ground_truth_contexts']]
retrieved_docs = [item['predicted_context'] for item in predicted_data]
ground_truth_answers = [answer for answer in qacs['ground_truth_answers']]
rag_answers = [item['predicted_answer'] for item in predicted_data]
questions = [question for question in qacs['questions']]

In [24]:
print(len(ground_truth_docs))
print(len(ground_truth_docs[0]))
print(len(retrieved_docs))
print(len(retrieved_docs[0]))

300
1
300
5


In [25]:
eval_results = eval_pipeline.run(
    {
        "mrr_evaluator": {"ground_truth_documents": ground_truth_docs,
	                          "retrieved_documents": retrieved_docs},
        "map_evaluator": {"ground_truth_documents": ground_truth_docs,
	                          "retrieved_documents": retrieved_docs},
        "recall_evaluator": {
            "ground_truth_documents": list(ground_truth_docs),
            "retrieved_documents": list(retrieved_docs),
        },
        "sas_evaluator": {"predicted_answers": rag_answers, "ground_truth_answers": list(ground_truth_answers)},
    }
)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [26]:
from haystack.evaluation.eval_run_result import EvaluationRunResult

inputs = {
    "question": list(questions),
    "contexts": list(ground_truth_docs),
    "answer": list(ground_truth_answers),
    "predicted_answer": rag_answers,
}

evaluation_result = EvaluationRunResult(run_name="pubmed_rag_pipeline", inputs=inputs, results=eval_results)
metrics, scores = evaluation_result.aggregated_report()['metrics'],evaluation_result.aggregated_report()['score']
haystack_eval_results = dict(zip(metrics, scores))
print(haystack_eval_results)

{'map_evaluator': 0.0, 'mrr_evaluator': 0.0, 'recall_evaluator': 0.0, 'sas_evaluator': np.float64(0.6521599508511523)}


### save system outputs


In [27]:
# creating an evaluation dataset
ds = []
for i in range(len(questions)):
    ds.append(
        {
            "user_input": questions[i],
            "retrieved_contexts":[d.content for d in retrieved_docs[i]],
            "response":rag_answers[i],
            "reference":ground_truth_answers[i]
        }
    )
with open('predicted_data.json','w') as jsonfile:
  json.dump(ds,jsonfile)


In [28]:
# creating an evaluation dataset
haystack_evals = {'haystack_evals':haystack_eval_results}
with open('haystack_evaluations.json','w') as jsonfile:
  json.dump(haystack_evals,jsonfile)
